In [14]:
from delta.tables import *

StatementMeta(, f5d76867-4c45-4472-aa2b-0c520cf959e3, 25, Finished, Available)

In [15]:
%run _Config

StatementMeta(, f5d76867-4c45-4472-aa2b-0c520cf959e3, -1, Finished, Available)

In [16]:
%run _WondeStructs

StatementMeta(, f5d76867-4c45-4472-aa2b-0c520cf959e3, -1, Finished, Available)

In [17]:
watermark_schema = StructType([
    StructField("school_id", StringType(), nullable=True),
    StructField("session_attendance_last_update", TimestampType(), nullable=True),
])

StatementMeta(, f5d76867-4c45-4472-aa2b-0c520cf959e3, 37, Finished, Available)

In [18]:
dfEmpty = spark.createDataFrame(spark.sparkContext.emptyRDD(), watermark_schema)

StatementMeta(, f5d76867-4c45-4472-aa2b-0c520cf959e3, 38, Finished, Available)

In [19]:
(dfEmpty.write
    .mode("overwrite")
    .option("mergeSchema", "true")
    .format("delta")
    .save(BronzeBasePath + "/Watermarks")
    )

StatementMeta(, f5d76867-4c45-4472-aa2b-0c520cf959e3, 39, Finished, Available)

In [20]:
deltaWatermarks = DeltaTable.forPath(spark, f"{BronzeBasePath}/Watermarks")
deltaSchools = DeltaTable.forPath(spark, f"{BronzeBasePath}/Schools")

StatementMeta(, f5d76867-4c45-4472-aa2b-0c520cf959e3, 40, Finished, Available)

In [21]:
dfUpdates = deltaSchools.toDF()

(deltaWatermarks
    .alias("watermarks")
    .merge(
        dfUpdates.alias("updates"), "watermarks.school_id = updates.school_id"
    )
    .whenNotMatchedInsert(
        values =
        {
            "school_id": "updates.school_id"
        })
    .execute()
)


StatementMeta(, f5d76867-4c45-4472-aa2b-0c520cf959e3, 41, Finished, Available)

In [22]:
df = spark.sql("SELECT * FROM Bronze.Watermarks LIMIT 1000")
display(df)

StatementMeta(, f5d76867-4c45-4472-aa2b-0c520cf959e3, 42, Finished, Available)

SynapseWidget(Synapse.DataFrame, 62342bd8-9763-46da-9db5-bf624cb12e9a)

In [23]:
df = spark.sql("SELECT * FROM Bronze.Schools LIMIT 1000")
display(df)

StatementMeta(, f5d76867-4c45-4472-aa2b-0c520cf959e3, 43, Finished, Available)

SynapseWidget(Synapse.DataFrame, 38064c12-3543-4b1e-9294-33b80f6245f4)

In [24]:
%%sql
UPDATE Bronze.Watermarks 
SET 
    session_attendance_last_update = '2022-08-31 08:00:00'
WHERE
    session_attendance_last_update IS NULL
-- school_id <> 'A1930499544'


StatementMeta(, f5d76867-4c45-4472-aa2b-0c520cf959e3, 44, Finished, Available)

<Spark SQL result set with 1 rows and 1 fields>